在SGD优化中，通常要设置一个learning rate(lr)，这个lr 太大不好，太小也不好，太大容易越过最低点而无法下降，太小了收敛太慢
![](http://cs231n.github.io/assets/nn3/learningrates.jpeg)

在cifar10数据集上，尝试了一下lr:`[10, 5, 1, 0.1, 0.01, 0.001, 0.0001, 0.00001]`，每种lr进行了1000个epoch迭代，网络使用了较为简单的LeNet网络。

从训练结果来看，**lr=10** 和 **lr=5** 这种特别大的lr情况下，在第一个epoch中前面几十个batchloss就会暴涨，超过最大值，最终出现NAN的情况。

In [28]:
df['train_lr_10'][15:35]

15    2.540980e+00
16    2.426146e+00
17    2.351841e+00
18    1.845384e+01
19    2.158724e+01
20    5.895906e+00
21    6.195771e+00
22    7.015497e+00
23    2.514503e+01
24    6.082247e+00
25    4.786431e+00
26    5.093921e+00
27    5.260998e+00
28    4.673571e+00
29    5.651251e+00
30    5.188484e+00
31    4.449006e+00
32    1.332958e+01
33    3.456380e+12
34             NaN
Name: train_lr_10, dtype: float64

---
**lr=1**时，也是属于非常大的情况，可以发现loss几乎不变，而**lr=0.00001**又是另一种极端情况，在1000个epoch中，loss收敛的非常慢，而**lr=0.01, lr=0.001, lr=0.0001**这几种情况相对较其他几种极端情况有明显的下降趋势。

In [9]:
from bokeh.plotting import figure, output_notebook, show
import pandas as pd
df = pd.read_csv('learning_rate.csv')

output_notebook()
p = figure(tools="pan,box_zoom,reset,save", 
    title="different learning rate (train)", x_axis_label='epoch', y_axis_label='loss',
    plot_width=900#     y_range=[0, 3],
)

p.line(df['epoch'], df['train_lr_1'], line_color='blue', line_width=2, legend="lr=1")
p.line(df['epoch'], df['train_lr_0.01'], line_color="red",line_width=2, legend="lr=0.01")
p.line(df['epoch'], df['train_lr_0.001'], line_color="green",line_width=2, legend="lr=0.001")
p.line(df['epoch'], df['train_lr_0.0001'], line_color="brown", line_width=2, legend="lr=0.0001")
p.line(df['epoch'], df['train_lr_0.00001'], line_color="black", line_width=2, legend="lr=0.00001")
p.legend.location = "top_right"
p.legend.click_policy="hide"
show(p)

Loading BokehJS ...

再来看看训练数据集（train）与测试数据集（val）的loss，因为使用的是比较简单的网络，没有用dropout, 也没有data augmentation因此出现了过拟合的现象。

In [19]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()
p = figure(
    tools="pan,box_zoom,reset,save", title="different learning rate (train)", x_axis_label='epoch', y_axis_label='loss',
    plot_width=900
)
p.circle(df['epoch'], df['train_lr_0.01'],color='red', size=2, legend="lr=0.01(train)")
p.circle(df['epoch'], df['val_lr_0.01'], color='blue', size=2, legend="lr=0.01(val)")
p.square(df['epoch'], df['train_lr_0.001'], color='red',size=2, legend="lr=0.001(train)")
p.square(df['epoch'], df['val_lr_0.001'], color='blue', size=2, legend="lr=0.001(val)")
p.triangle(df['epoch'], df['train_lr_0.0001'], color='red',size=2, legend="lr=0.0001(train)")
p.triangle(df['epoch'], df['val_lr_0.0001'], color='blue', size=2, legend="lr=0.0001(val)")

p.legend.location = "top_right"
p.legend.click_policy="hide"
show(p)

Loading BokehJS ...

---
再来看看通过通过动态调整lr的情况，这里建档的使用